In [ ]:
from torchvision.datasets import CIFAR10
import numpy as np
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import numpy as np

In [ ]:
SPLIT=True
MODEL_ID=2
BATCH_SIZE=2**9

In [ ]:
train_dataset = CIFAR10('.', train=True, transform=ToTensor(), download=True)
valid_dataset = CIFAR10('.', train=False, transform=ToTensor())

if SPLIT:
    labels = np.array(train_dataset.targets)
    condition = labels < 5 if MODEL_ID == 1 else labels >= 5
    train_dataset = Subset(train_dataset, condition.nonzero()[0])


train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=512)

In [ ]:
from torchvision.models import resnet18
model = resnet18(num_classes=10)

In [ ]:
from torch.optim import SGD
optimizer = SGD(model.parameters(), lr=1e-3)

In [ ]:
from torch import nn
loss_fn = nn.CrossEntropyLoss()

In [ ]:
import torch
def train():
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = loss_fn(outputs, target)
        loss.backward()
        optimizer.step()
    return loss.item()

def validate():
    model.eval()
    total_samples = 0
    val_score = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        pred = outputs.argmax(dim=1)
        val_score += pred.eq(target).sum().cpu().numpy()
        total_samples += len(target)
    return val_score / total_samples

for epoch in range(20):
    loss = train()
    acc = validate()
    print(f'Epoch {epoch+1}:\tTrain loss:{loss:4f}\tTest accuracy:{acc:4f}')

In [ ]:
import pickle
with open(f'model_{MODEL_ID}.pkl','wb') as f:
    pickle.dump({n: p.detach().numpy() for n,p in model.named_parameters()}, f)    